In [1]:
import random
import numpy as np
import aifc
import csv
import pandas as pd
import sys

def extract_file():
    df = pd.read_csv('train.csv', header=0, sep=',')
    df.columns=['sample', 'label'] 

def pourc(tl, pourc):
    return pourc*tl/100

def create_label():
    label_list = []
    for i in range(len(df)):
        label_list.append(df['label'][i])
    return label_list

def split_our_lists():
    extract_file()
    train = []
    validation = []
    test = []
    random.shuffle(create_label())
    tl = len(label_list)
    
    for i in range(tl):
        if i < pourc(tl,60):
            train.append(label_list[i])
        elif i < pourc(tl,80):
            validation.append(label_list[i])
        else:
            test.append(label_list[i])
    return len(train), len(validation), len(test)

split_our_lists()

NameError: name 'df' is not defined